In [12]:
# Package that provides access to Fits files 
from astropy.io import fits     
import numpy
import OpenEXR
import Imath
from PIL import Image

In [13]:
def exr_to_fits(input_exr, output_fits):
    
    exr = OpenEXR.InputFile(input_exr)
    exr_bytes = exr.channel("R", Imath.PixelType(Imath.PixelType.FLOAT))
            
    exr_image = Image.frombytes("F", (2048, 1489), exr_bytes)
    
    exr_array = numpy.array(exr_image)
    
    print (exr_array[-1])
                    
    hdu = fits.PrimaryHDU(exr_array)
    hdul = fits.HDUList([hdu])
    hdul.writeto(output_fits)
    
    return pix
    
    
    

In [14]:

def fits_to_exr(input_fits, output_exr32):
    
    # Returns HDUList object which is a list-like collection of HDU objects.
    # Each HDU contains a header and data. Header contains keyword, value, and comment. Keyword / value
    # can be manipulated like a python dict. Data is a numpy ndarray (n-dimensional array). 
    # For now, we only look at the first HDU (this should be changed later). 
    HDUList = fits.open(input_fits)
    fits_header = HDUList[0].header 
    fits_data = HDUList[0].data
    HDUList.close()
    
    # Get the dimension of the image. 
    width, height = fits_data.shape
    print (width, height)
    
    # Convert the elements in fitz_data to float32 format (presumably from float64, but this should be checked).
    # This can be done analogously for 16 bit.    
    fits_data_f32 = numpy.float32(fits_data)
    
    print (fits_data_f32[-1])

    # Create the EXR header with appropriate height and width.
    # Copy the Fits header to EXR. Probably need to do more of a deep copy later. 
    header = OpenEXR.Header(height, width)
    header['header'] = fits_header
    
    # Create EXR output file with header. Write Fits data to EXR red channel. 
    EXR32 = OpenEXR.OutputFile(output_exr32, header)
    EXR32.writePixels({"R": fits_data_f32})
    
    return fits_data_f32
    

In [15]:
fits_input = "/home/idies/workspace/Storage/mtaghiza/exr/SkyServer FITS/2MASS J11582332-0020114/frame-r-006793-3-0127.fits"
exr_output = "exr_demo.exr"
print ("FLAG")
one = fits_to_exr(fits_input, exr_output)
print ("FLAG1")
two = exr_to_fits("exr_demo.exr", "fits_demo7.fits")

for i in range(len(one)):
    for j in range(len(one[i])):
        if one[i][j] != two[i][j]:
            print ("Ouch")
            break
print ("The images are the same!")

FLAG
1489 2048
<class 'numpy.ndarray'>
[-0.02185059 -0.01647949  0.05352783 ... -0.05371094  0.01580811
  0.00512695]
[-0.03265381  0.01583862  0.02120972 ... -0.059021   -0.00556946
 -0.00556946]
[ 0.00611877 -0.01004028  0.02224731 ...  0.02218628  0.01147461
  0.02752686]
FLAG1
[ 0.00611877 -0.01004028  0.02224731 ...  0.02218628  0.01147461
  0.02752686]


OSError: File 'fits_demo7.fits' already exists.